In [ ]:
!pip install spacy
!pip install numpy==1.26.4
!python -m spacy download en_core_web_sm

In [5]:
import spacy

nlp = spacy.load("en_core_web_sm")

with open("../Disertatie/Dataset/Lyrics_ExtractKLLyrics_29.05.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for song in data:
    lyrics = song["lyrics"]
    doc = nlp(lyrics)
    
    lemmas = [token.lemma_ for token in doc]
    lemmas_string = " ".join(lemmas)
    song["lyrics"] = lemmas_string

with open("../Disertatie/Dataset/Lyrics_Lemmatization_31.05.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

Topic of the songs using LDA

In [ ]:
!pip install scikit-learn
!pip install matplotlib

In [10]:
!pip install pronouncing
!pip install tensorflow

   ---------------------------------------- 0.0/375.9 MB ? eta -:--:--
   ---------------------------------------- 1.8/375.9 MB 12.6 MB/s eta 0:00:30
   - -------------------------------------- 12.8/375.9 MB 38.4 MB/s eta 0:00:10
   -- ------------------------------------- 24.4/375.9 MB 42.9 MB/s eta 0:00:09
   --- ------------------------------------ 37.5/375.9 MB 49.6 MB/s eta 0:00:07
   ----- ---------------------------------- 52.7/375.9 MB 54.2 MB/s eta 0:00:06
   ------- -------------------------------- 66.8/375.9 MB 56.9 MB/s eta 0:00:06
   -------- ------------------------------- 80.0/375.9 MB 58.0 MB/s eta 0:00:06
   --------- ------------------------------ 93.3/375.9 MB 59.0 MB/s eta 0:00:05
   ---------- ---------------------------- 105.6/375.9 MB 59.2 MB/s eta 0:00:05
   ----------- --------------------------- 111.9/375.9 MB 56.3 MB/s eta 0:00:05
   ------------ -------------------------- 119.0/375.9 MB 53.5 MB/s eta 0:00:05
   ------------- ------------------------- 131.1/3

In [4]:
import torch
import pronouncing
import re

def get_last_word(line):
    words = re.findall(r"\b\w+\b", line.lower())
    return words[-1] if words else None

def lines_rhyme(word1, word2):
    if not word1 or not word2:
        return False
    rhymes = pronouncing.rhymes(word1)
    return word2 in rhymes or word1 == word2

def generate_lyrics_with_rhyme(prompt, rhyme_scheme='AABB', num_verses=8, max_retries=5):
    generated_lyrics = []
    current_prompt = prompt.strip() + "\n"

    for i in range(0, num_verses, 2):
        inputs = tokenizer_gpt2(current_prompt, return_tensors="pt").to(device)
        output_sequences = model_gpt2.generate(
            **inputs,
            max_length=inputs['input_ids'].shape[1] + 30,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.95,
            repetition_penalty=1.2,
            pad_token_id=tokenizer_gpt2.eos_token_id,
            num_return_sequences=1,
        )
        generated_text = tokenizer_gpt2.decode(output_sequences[0], skip_special_tokens=True)
        new_verse1 = generated_text[len(current_prompt):].strip().split('\n')[0]
        generated_lyrics.append(new_verse1)
        current_prompt += new_verse1 + "\n"

        last_word1 = get_last_word(new_verse1)

        for attempt in range(max_retries):
            inputs = tokenizer_gpt2(current_prompt, return_tensors="pt").to(device)
            output_sequences = model_gpt2.generate(
                **inputs,
                max_length=inputs['input_ids'].shape[1] + 30,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=0.95,
                repetition_penalty=1.2,
                pad_token_id=tokenizer_gpt2.eos_token_id,
                num_return_sequences=1,
            )
            generated_text = tokenizer_gpt2.decode(output_sequences[0], skip_special_tokens=True)
            new_verse2 = generated_text[len(current_prompt):].strip().split('\n')[0]

            last_word2 = get_last_word(new_verse2)
            if lines_rhyme(last_word1, last_word2):
                generated_lyrics.append(new_verse2)
                current_prompt += new_verse2 + "\n"
                break
            elif attempt == max_retries - 1:
                generated_lyrics.append(new_verse2)
                current_prompt += new_verse2 + "\n"

    return generated_lyrics

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm.notebook import tqdm

model_name = "../Disertatie/Training/kendrick-lyrics-gpt2/checkpoint-12036"
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(model_name)
model_gpt2 = GPT2LMHeadModel.from_pretrained(model_name).eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_gpt2.to(device)

prompt = "Growing up was never easy, too much pain to digest,\n"
lyrics = generate_lyrics_with_rhyme(prompt, num_verses=8)

for i, verse in enumerate(lyrics, 1):
    print(f"Verse {i}: {verse}")

Verse 1: |Cause that's my life now (Yeah) Life is tough though you've been living it since birthdays for years! No? Haha (
Verse 2: ’Mm.A., bitch!) Yes man-Man I'm fuckin' nasty! You can smell it (Huh?) K.Dot
Verse 3: (Kendrick Lamar) What the fuck do ya think 'bout a nigga got there but some big fat ass gang members (Uh huh
Verse 4: Uh huh  What if y”all just livin'? Heesh yeah and let me show them somethin', baby boy,
Verse 5: you know the truth right here (Yah), I know (You ain't talkin') shit (Haha) Uh huh what the fiends
Verse 6: who hate us donn't love us no more (Uh huh what they bout with yo niggas) So I'll let him rap
Verse 7: I'll let his mother eat like John Madden (Oh yeah) What else do he have? Baby step on em daddy line by toe ('Treat
Verse 8: em right) Uh huh (Woo hoo ho) Alright I got one (Yep) And when we finally come around, she gotta get


In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np


model_lstm = load_model("../Disertatie/Training/kendrick-lyrics-lstm/lstm_kendrick_model.h5")
with open('../Disertatie/Training/kendrick-lyrics-lstm/tokenizer_kendrick.pkl', 'rb') as f:
    tokenizer_lstm = pickle.load(f)

max_seq_len=2203
seed_text = "Growing up was never easy, too much pain to digest,\n"
generated_verses = generate_lyrics_with_rhyme(seed_text, num_verses=8)

for i, verse in enumerate(generated_verses, 1):
    print(f"Verse {i}: {verse}\n")

Verse 1: |it’s been so far away from me since I grew up and this is no new reality for you young men of the world (Yeah

Verse 2: I'ma gettin' some relief) So what are y'all doin'? Tell 'em I wanna stay with my family now baby girl

Verse 3: (Uh huh), tell them I'm goin' straight for ya lil bitch that got a son in her bed right now momma keep an eye

Verse 4: Wow (Woah) Now fuck yo nigga, tryna figure it out (Ah! Baby boy!) But still, they love me now I

Verse 5: That's just how I feel right here (Woah-woah) Yeah (Baby, man) Yeah (Baby homie) Yeah yeah (

Verse 6: And even though she be livin', baby boom) Alright (Wooh oh) Alright (Ooh wooh – yeah) Yeah [–]

Verse 7: You know what? They love me now (Hey) You know what we do want again (Woah-woow) So what are us dy

Verse 8: Now let us talk about your mother life at least once every two weeks (Woah-woosh) Mm hmm (Yeah) Man,

